## LukG preproduction analysis

In [ ]:
# -- IMPORTS --
# - mdanalysis -
import MDAnalysis as mda
from MDAnalysis.lib import distances
from MDAnalysis.analysis.rms import RMSD, rmsd, RMSF
from MDAnalysis.analysis import diffusionmap
from pathlib import Path

# - plotting -
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# - clustering -
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import AgglomerativeClustering
from collections import Counter

# - general -
import numpy as np
import pandas as pd
import pickle
import math

### Import files 

In [ ]:
# -- Make UNIVERSES --
# data/gromacs/
simdir = Path("../data/")
rdir = simdir / "gromacs/" # here are the results of analyses made with gromacs

#imdir = Path("../figures/")
imdir = Path("../figures/png_figs/")

lukG_GRO = simdir/ 'lukG.gro'

lukG_XTC = simdir /'trajectory_noPBC.xtc'

lukG_XTC_al = simdir / 'trajectory_noPBC_alignedCA.xtc'

#the topology file contains a list of atoms (gromacs gro in our case)
#the trajectory file provides coordinates that are ordered in the same way as the list of atoms in the topology (MD trajectory Gromacs XTC)

The trajectory `trajectory_noPBC.xtc` was made with gromacs <i>trjconv</i> with the command: <br>
```gmx trjconv -s step7_final.tpr -f step7_final.xtc -pbc mol -center -o trajectory_noPBC.xtc```<br>
<br>
The trajectory `trajectory_noPBC_alignedCA.xtc` is the same as the previous, but aligned over the Alpha Carbons with MDAnalysis <i>AlignTraj</i>. <br>
You can see how a few blocks below. 

### Check trajectory INFO

In [ ]:
# load the simple universe, NOT aligned.
lukG = mda.Universe(str(lukG_GRO), str(lukG_XTC))

print("The number of frames is", lukG.trajectory.n_frames)
#print(lukG.select_atoms("protein").residues)
print("The number of residues is", len(lukG.select_atoms("protein").residues))

#lukG.atoms.dimensions

# Also check the same info in the aligned trajectory
lukG_al = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))
print("[aligned] The number of frames is", lukG_al.trajectory.n_frames)
print("[aligned] The number of residues is", len(lukG_al.select_atoms("protein").residues))

In [ ]:
# We can see that the counting of the aminoacids starts at 32 and ends at 338
lukG_al.select_atoms("protein").residues[0], lukG_al.select_atoms("protein").residues[-1]

# Giving us a total of 307 aminoacids in the protein

In [ ]:
# We used this function to only make the images for the report,
# only if they were not already present in the figures folder
# imdir = Path("../figures/")

def make_image(fig_obj, output_name):
    # check if image exists with output_name
    if (imdir / str(output_name + '.png')).exists():
        return None #"Image already exists, skipping!"
  
    else:
        return plotly.io.write_image(fig_obj, str(imdir / output_name) + '.png', format='png')

# dummy func so that you don't save images
# def make_image(fig_obj, output_name):
#     return None

### Density plot
To build this plot we used three commands with gromacs, to compute each one of the files that are being read in the first three lines (variables `popc`, `prot`, and `tipp`). <br>
Specifically, we used the command:
```
gmx_mpi density -f trajectory_noPBC_alignedCA.xtc -s step7_final.tpr -o density.xvg
```
And selected the groups of atoms we were interested in: Protein [1], Water/TIP3 [17], Membrane/POPC [14].

In [ ]:
# import the data
popc = np.genfromtxt(rdir / "density/POPC_density.xvg", usecols = (0,1), skip_header = 24)
prot = np.genfromtxt(rdir / "density/protein_density.xvg", usecols = (0,1), skip_header = 24)
tipp = np.genfromtxt(rdir / "density/TIP3_density.xvg", usecols = (0,1), skip_header = 24)

# plot
fig = go.Figure()
fig.add_trace(go.Line(x = tipp[:,0], y = tipp[:,1], name = "TIP3"))
fig.add_trace(go.Line(x = prot[:,0], y = prot[:,1], name = "Protein"))
fig.add_trace(go.Line(x = popc[:,0], y = popc[:,1], name = "POPC"))


fig.update_layout(title={'text':"Density plot of LukG",
                         'x':0.5,
                         'y':0.88,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis = {'title': "z axis [nm]"},
                  yaxis = {'title': "density [Kg / m³]"})



fig.show()
make_image(fig, "density_plot")

### PCA

In [ ]:
#rec_pca = PCA(lukG_al, select = 'name CA').run()

# Run only if there isn't a pcadata file, and make it.
# if present, load it
if not (simdir / "pcadata.pickle").exists():
    rec_pca = PCA(lukG_al, select = 'name CA').run()
   
    with open(simdir / "pcadata.pickle", "wb") as outf:
        pickle.dump(rec_pca, outf)
else:
    with open(simdir / "pcadata.pickle", "rb") as infile:
        rec_pca = pickle.load(infile)


# limiting the analysis to the first i components for a clearer graph
i = 20

#xdata = np.arange(rec_pca.cumulated_variance.shape[0])
xdata = np.arange(i)
ydata = rec_pca.cumulated_variance

fig = go.Figure()
fig.add_trace(go.Scatter(x = xdata, y = ydata[:i],
                         mode = 'lines+markers'))

fig.update_layout(title={'text':"PCA analysis",
                         'x':0.5,
                         'y':0.87,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis_title="Components",
                  yaxis_title="Cumulated variance explained")

fig.show()
make_image(fig, "pca_cumulated_variance")

In [ ]:
#pca_space = rec_pca.transform(lukG_al.select_atoms("name CA"),3)

# save pca_space, it takes a while to compute
if not (simdir / "pcaspace.pickle").exists():
    pca_space = rec_pca.transform(lukG_al.select_atoms("name CA"),3)
   
    with open(simdir / "pcaspace.pickle", "wb") as outf:
        pickle.dump(pca_space, outf)
else:
    with open(simdir / "pcaspace.pickle", "rb") as infile:
        pca_space = pickle.load(infile)


pca_data = pd.DataFrame(pca_space, columns = ['first_comp', 'second_comp', 'third_comp'])

fig = px.scatter_3d(pca_data, x = 'first_comp', y = 'second_comp', z = 'third_comp',
                   color = pca_data.index.values)

fig.update_traces(marker_size = 2)

# change zoom and angle of the view
fig.update_layout(scene_camera = dict( eye = dict(
                  x = 1.8,
                  y = 1.5,
                  z = 0.5))
                 )
fig.show()
make_image(fig, "pca_scatter")

We also wanted to observe only the last second half of the trajectory in the reduced PCA space, so we decided to plot the last 3000 frames:

In [ ]:
fig = px.scatter_3d(pca_data[3000:], x = 'first_comp', y = 'second_comp', z = 'third_comp',
                   color = pca_data.index.values[3000:])

fig.update_traces(marker_size = 2)

# change zoom and angle of the view
fig.update_layout(scene_camera = dict( eye = dict(
                  x = 1.8,
                  y = 1.5,
                  z = 0.5))
                 )
fig.show()

make_image(fig, "pca_scatter_secondhalf")

### Calculate RMSD before and after the alignment
Here we are only computing the RMSD between the first and last frame of the trajectory, to check that the alignment actually reduces this value as it should.

In [ ]:
# Function to compute simple rmsd between two frames
def test_rmsd(trj1, trj2):
    """Tests rmsd between the first frame of trj1 and last of trj2"""
    trj1.trajectory[0]
    trj2.trajectory[-1]
    
    t1_ca = trj1.select_atoms("name CA")
    t2_ca = trj2.select_atoms("name CA")
    
    return rmsd(t1_ca.positions, t2_ca.positions, superposition=False)


In [ ]:
# Let's try on the NON ALIGNED trajectory
lukG = mda.Universe(str(lukG_GRO), str(lukG_XTC))
ref = mda.Universe(str(lukG_GRO), str(lukG_XTC))

print(test_rmsd(lukG, ref))

Now we can align the trajectory to itself in the first frame (res).

In [ ]:
# No need to re-run this block every time.
# Only run if the file doesn't exist in the ../data/ directory
if not (simdir / "trajectory_noPBC_alignedCA.xtc").exists():
    from MDAnalysis.analysis import align

    lukG = mda.Universe(str(lukG_GRO), str(lukG_XTC))
    ref = mda.Universe(str(lukG_GRO), str(lukG_XTC))

    aligner = align.AlignTraj(lukG, ref, select = 'name CA',
                    filename = '../data/trajectory_noPBC_alignedCA.xtc').run()
    
    # load a new universe with this trajectory
    lukG_al = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))
    
else:
    print("Aligned trajectory already exists, skipping...")

Then we can check if the RMSD changed. <br>
We can also see if the two different alignments give different results. <br>
Specifically, we expect a decreased RMSD, thanks to the alignment.

In [ ]:
# reload the new aligned trajectories, and the reference
lukG_al = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))
ref = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))

# NOTE: even if we are loading only the aligned version, the first frame is the same between the two.

# print rmsd between first and last frame
print(test_rmsd(lukG_al, ref))

We can see that the RMSD diminished by a lot.

### RMSD of the whole trajectory with a single frame as reference

In [ ]:
# If possible, load from memory
if not (simdir / "rmsd.pickle").exists():
    # reload the universes
    lukG = mda.Universe(str(lukG_GRO), str(lukG_XTC))
    lukG_al = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))

    # run the Rmsd over lukG_al with first frame of lukG_al as reference
    rmsd_arr2 = RMSD(lukG_al.select_atoms("name CA"), lukG_al.select_atoms("name CA")).run()

    # make a pandas dataframe of the data for ease of use
    rmsd_df =  pd.DataFrame(rmsd_arr2.rmsd, columns=['idx','time','CA'])
   
    with open(simdir / "rmsd.pickle", "wb") as outf:
        pickle.dump(rmsd_df, outf)
else:
    with open(simdir / "rmsd.pickle", "rb") as infile:
        rmsd_df = pickle.load(infile)

In [ ]:
# plot the RMSD values
y = rmsd_df["CA"]
x = rmsd_df["idx"]

fig = px.line(x = x, y = y,
              labels={
                  'x':'Frame number',
                  'y':'RMSD value [Å]'
                  })

# Set title and position
fig.update_layout(
    title={
        'text':"LukG RMSD analysis, CA aligned",
        'x':0.5,
        'y':0.95,
        'xanchor':'center',
        'yanchor':'top'
    })

fig.show()
make_image(fig, "rmsd_plot")

### Block Analysis
We implemented a Block analysis to understand how we could implement a faster RMSD Matrix while keeping the information of our system.

In [ ]:
# Find the total number of frames
n_frames = lukG_al.trajectory.n_frames

# range 2 -> 3000
nran = np.arange(2, n_frames//2+1)

# array of the remainders between nframes and nran
divisionRest = (n_frames-1) % nran

# we only keep the perfect dividors
# so here we have a list of the number of frames
# we will use in each block, for each block analysis
frames_x_block = nran[np.where(divisionRest == 0)]

# in one line
#frames_x_block_v2 = [x for x in np.arange(2, n_frames//2+1) if (n_frames-1)%x == 0]

In [ ]:
# Get the references for index and time
idx_ref = rmsd_df.idx.values[1]
time_ref = rmsd_df.time.values[1]

# Set the empty variables
Nc_val = []
Time_val = []
RMSD_val = []

for i in frames_x_block:
    Nc = n_frames//i
    Nc_val.append(Nc)
    # splits the rmsd data in as many sub-arrays as Nc
    splits = np.array_split(rmsd_df.CA.values[0:6000], Nc)
    
    splits_avg = np.sum(splits, axis = 1)/i
    splits_RMSD = np.std(splits_avg, dtype=np.float64)
    
    RMSD_val.append(splits_RMSD)
    Time_val.append(Nc*time_ref/idx_ref)


In [ ]:
print("M Values: {}".format(frames_x_block))
print("\nNc Values: {}".format(Nc_val))
print("\nTime Values: {}".format(Time_val))
print("\nRMSD Values: {}".format(RMSD_val))

In [ ]:
# plotting the Block Analysis
fig = go.Figure()

# removed last 5 entries for clarity (values are descending)
fig.add_trace(go.Scatter(x = Nc_val[6:],
                         y = RMSD_val[6:],
                         mode = 'lines+markers'))

# Set title and position
fig.update_layout(
    title={
        'text':"Block Analysis: CA",
        'x':0.5,
        'y':0.89,
        'xanchor':'center',
        'yanchor':'top'
    }, 
    xaxis_title = "Blocks (Nc)",
    yaxis_title = "RMSD Values")


fig.show()
make_image(fig, "block_analysis_plot")

### RMSD - MATRIX
We then compute the RMSD matrix using a frame every 15, as they should be uncorrelated. <br>
Check these links: <br>
- [blog_help](https://groups.google.com/g/mdnalysis-discussion/c/1iBwPqaAGLg?pli=1)
- [MDA_manual](https://userguide.mdanalysis.org/stable/examples/analysis/alignment_and_rms/pairwise_rmsd.html)

In [ ]:
# Load a new universe with a subset of frames 

# restricted universe every n frames
# for in_memory_step to work, in_memory MUST BE == True!!
n = 15
u_clst = mda.Universe(str(lukG_GRO), str(lukG_XTC_al), in_memory = True, in_memory_step =  n)

In [ ]:
# test of diffusionmap for pairwise RMSD of trj with itself
# this computes the matrix in an optimized way.
# (we could probably take even more frames than 1 every 15 and still
# run this in a reasonable amount of time ~minutes)
rmsd_matrix = diffusionmap.DistanceMatrix(u_clst, select='name CA').run()

In [ ]:
# Plot the resulting matrix

prmsd_fig = px.imshow(rmsd_matrix.results.dist_matrix, aspect = 'equal')

prmsd_fig.update_layout(title={
        'text':"RMSD Matrix",
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    }, 
    xaxis_title = "Frame number",
    yaxis_title = "Frame number")

prmsd_fig.show()

make_image(prmsd_fig, "rmsd_matrix_plot")

We want to check that this is an actual pairwise implementation of the RMSD by extracting the data regarding all frames vs. first frame (as we did previously in the RMSD blocks), and plotting them together.

In [ ]:
# Extract the relevant data
m_arr = rmsd_matrix.dist_matrix[1, :]

# Scale the data from the pairwise, as it is 1 frame every 15
# scale = 6001 / 401

fig = go.Figure()

fig.add_trace(go.Line(y = rmsd_df.iloc[:,2], opacity = 0.2, name = "Single RMSD"))
fig.add_trace(go.Line(y = m_arr, x = (x * (6001 / 401)), name = "Pairwise-RMSD"))

fig.update_layout(title={
        'text':"RMSD vs Pairwise-RMSD",
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    }, 
    xaxis_title = "Frame number",
    yaxis_title = "RMSD value [Å]")

fig.show()
make_image(fig, "rmsd_vs_singlePairwise")

As expected, the Pairwise-RMSD data is smoother (as some points are omitted), but we can see that it closely resembles the Single RMSD data we previously computed.

### Hierarchical clustering 

In [ ]:
# We tested different linkage measures
#Z = linkage(rmsd_matrix.dist_matrix, 'average')
#Z = linkage(rmsd_matrix.dist_matrix, 'weighted')
Z = linkage(rmsd_matrix.dist_matrix, 'ward')

X = dendrogram(Z)

fig = ff.create_dendrogram(Z)

fig.update_layout(width=900, height=600)
fig.show()

make_image(fig, "dendrogram_image")

### RMSF 

In [ ]:
if not (simdir / "rmsf.pickle").exists():
    ca_atoms = lukG_al.select_atoms("protein and name CA") # this is the AtomGroup
    myRMSF = RMSF(ca_atoms).run()
   
    with open(simdir / "rmsf.pickle", "wb") as outf:
        pickle.dump(myRMSF, outf)
else:
    with open(simdir / "rmsf.pickle", "rb") as infile:
        myRMSF = pickle.load(infile)


In [ ]:
data = myRMSF.rmsf
x = np.arange(len(data))

fig = px.line(x = x, y = data, 
              labels={'x':"Residue index",
                      'y':"RMSF value [Å]"}
             )

# Set title and position
fig.update_layout(
    title={
        'text':"LukG RMSF analysis",
        'x':0.5,
        'y':0.95,
        'xanchor':'center',
        'yanchor':'top'
    })

# Add colorful bars :)

orange = "#FF681F"
seagreen = "#2E8B57"

fig.add_vrect(x0 = 253, x1 = 257, fillcolor="seagreen",
 opacity = 0.25, line_width=1, line_color="green")


# the max we chose
fig.add_vrect(x0 = 42, x1 = 46, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 66, x1 = 70, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 130, x1 = 134, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 156, x1 = 160, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 174, x1 = 178, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 184, x1 = 188, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 236, x1 = 240, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")
fig.add_vrect(x0 = 261, x1 = 265, fillcolor="#FF681F",
 opacity = 0.25, line_width=1, line_color="red")



# NOTE Add legend pliz
# it's ugly, but I'm thinking of adding a "false trace"
# that doesn't hold any information but that can be used 
# for a legend. To do that I am using plotly graph objects!
# And I create the trace as a Bar for the square shape
# of the legend element.
fig.add_trace(go.Bar(x=[np.nan], y=[np.nan], 
                     marker_color = '#2E8B57',
                     opacity = 0.25, name = "Low areas"))
fig.add_trace(go.Bar(x=[np.nan], y=[np.nan], 
                     marker_color = '#FF681F', 
                     opacity = 0.25, name = "High areas"))

fig.show()
make_image(fig, "rmsf_plot")

In [ ]:
print(f"RMSF values of residues involved in the rim domain.\n E171: {round(data[171],2)}, D189: {round(data[189],2)}, D191: {round(data[191],2)}")

# analisi che vorrei fare, dire tipo quanti aminoacidi si muovono più di un certo treshold sul totale.
# giusto per dire "la proteina si muove poco in generale, solo il X% dei residui si muove più di 2 angstrom"
j = 2
print(f"Only {round(sum(data > j) / len(data),2)}% of the protein resiudes moves more than {j} Å.")

In [ ]:
# Some descriptive statistics of the RMSF data
pd.DataFrame(data).describe()

In [ ]:
# A look at some of the max and min regions in the graph
print(np.argmax(data[15:-5]) + 15, data[130:134]) # 132: 130 - 134
print(np.argmin(data), data[253:257])             # 255: 253 - 257

Here we wanted to exploit the RMSF data to find some atoms that were far apart and that moved a little during the simulation (low RMSF values), in order to use them in the analysis to check the angle of the protein with the z axis.

In [ ]:
# argmin in some subregions that had low values from the graph
print(np.argmin(data[0:40]))
print(np.argmin(data[260:300])+260)
#39 and 272

# to check the atom id to use in GROMACS
print(lukG_al.residues[39].atoms.select_atoms("name CA"))
print(lukG_al.residues[272].atoms.select_atoms("name CA"))


In [ ]:
# Compute the beta factor 
lukG_pdb = simdir / "LukG_AF2.pdb"
pdb_universe = mda.Universe(str(lukG_pdb))
ca_atoms_pdb = pdb_universe.select_atoms("protein and name CA")
#ix (index) -> unique code of the atom
betas = pdb_universe._topology.tempfactors.values[ca_atoms_pdb._ix]

In [ ]:
fig = make_subplots(specs = [[{"secondary_y" : True}]])

fig.add_trace(go.Scatter(y = -betas, name = "RMSF",
                         line = {'color':'red', 'width':2}),
             secondary_y=True)

fig.add_trace(go.Scatter(y = myRMSF.results.rmsf, name = "B-Factor",
                        line = {'color':'blue'}), secondary_y = False)

fig.update_layout(
    title={'text':"RMSF vs. Experimental Beta Factor",
        'x':0.5,
        'y':0.88,
        'xanchor':'center',
        'yanchor':'top'},
    xaxis_title = "Residue index",
    yaxis_title = "$\\beta$")

fig.update_yaxes(title_text = "RMSF value [Å]", secondary_y=False)
fig.update_yaxes(title_text = "B-Factor", secondary_y=True)

fig.show()
make_image(fig, "rmsf_beta_plot")

### Contact map

In [ ]:
ref = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))
Ca = ref.select_atoms('name CA')
d_CaCa = distances.distance_array(Ca.positions, Ca.positions)

In [ ]:
fig = px.imshow(d_CaCa)

fig.update_layout(
    title={
        'text':"Contact Map",
        'x':0.5,
        'y':0.95,
        'xanchor':'center',
        'yanchor':'top'
    },
    xaxis_title = "Residue index", yaxis_title= "Residue index")

fig.show()
make_image(fig, "contact_map")

### Radius of gyration

In [ ]:
# If data doesn't exist, compute it, otherwise load it
if not (simdir / "rgyr.data").exists():
    # remeber we loaded
    lukG_al = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))
    ca_atoms = lukG_al.select_atoms("protein and name CA")

    Rgyr = []
    for ts in lukG_al.trajectory:
        Rgyr.append(ca_atoms.radius_of_gyration())
    
    with open("../results/rgyr.data", "wb") as outfile:
        pickle.dump(Rgyr, outfile)
else:
    with open("../results/rgyr.data", "rb") as infile:
        Rgyr = pickle.load(infile)

In [ ]:
fig = px.line(Rgyr)

fig.update_layout(title={'text':"Radius of Gyration plot",
                         'x':0.5,
                         'y':0.95,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis_title = "Frame number",
                  yaxis_title = "Radius of Gyration [Å]",
                  showlegend = False)

fig.show()
make_image(fig, "radius_gyration_plot")

### Free energy Ramachandran plot

We computed the Ramachandran angles throughout the simulation with GROMACS, and then we plotted it by counting the number of times we saw a certain angle in the resulting file. <br>
We used the command: <br>
```gmx rama -f trj.xtc -s trj.tmp -o rama_output.xvg```

In [ ]:
# Load data from gromacs files
phi = np.genfromtxt(rdir / "rama_output.xvg", usecols=0, skip_header=34)+180
psi = np.genfromtxt(rdir / "rama_output.xvg", usecols=1, skip_header=34)+180

In [ ]:
# Create an empty matrix 
rama_matrix = np.zeros((361, 361))

# Count the occurrencies and populate the matrix
for f in range(0,len(phi), 306):
    for res in range(f, f+306):
        phi_rounded = int(round(phi[res]))
        psi_rounded = int(round(psi[res]))
        rama_matrix[psi_rounded-1,phi_rounded-1] +=1
        

In [ ]:
rama_df = pd.DataFrame(rama_matrix)
rama_df.columns = range(-180, 181)
rama_df.index = range(-180, 181)

rama_df.sort_index(axis=0, level=None, ascending=False, inplace=True)

In [ ]:
# Plot the resulting matrix as heatmap
# origin = '', sets which data to put in the origin
ax_rama = px.imshow(rama_df, origin='lower', color_continuous_scale='RdBu_r')

ax_rama.update_layout(title={'text':"Ramachandran Map",
                             'x':0.5,
                             'y':0.95,
                             'xanchor':'center',
                             'yanchor':'top'},
                      xaxis_title = "$\\phi \ [°]$",
                      yaxis_title = "$\\psi \ [°]$",
                      width = 800,
                      height = 800)

ax_rama.show()
make_image(ax_rama, "ramachandran_plot")

### Analysis of protein-membrane distance
Here we look at the distance on the z-axis between the center of mass of all the POPC residues and that of the protein.<br>
Note: This was made with the aligned trajectory.

In [ ]:
# For each frame in the trajectory,
# write to a file the distance between the centers of mass of
# the protein and all the POPC residues
if not (simdir / "protein-membrane_distance.csv").exists():
    with open(simdir / "protein-membrane_distance.csv", "w") as outfile:
        for ts in lukG.trajectory[:]:
            cpop = lukG.select_atoms("resname POPC").center_of_mass()
            cpro = lukG.select_atoms("protein").center_of_mass()
            outfile.write( ",".join(str(cpro - cpop)[1:-1].split()) + "\n")
else:
    print("File already exists, no need to recompute.")

In [ ]:
# load the previous file
difference_df = pd.read_csv(simdir / "protein-membrane_distance.csv", header = None)

fig = px.line(y = difference_df.iloc[:, 2])

fig.update_layout(title={'text':"<b>Protein-membrane distance with MDA</b>",
                         'x':0.5,
                         'y':0.95,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis_title = "Frame number",
                  yaxis_title = "Distance [Å]")

fig.show()
make_image(fig, "protein_membrane_distance_mda")

This is the result of the GROMACS <i>mindist</i> function between all the atoms of the protein and all the atoms of the POPC lipids.

In [ ]:
# mindist
mindist = np.genfromtxt(rdir / "mindist/mindist.xvg", usecols = (0,1), skip_header = 24)

fig = px.line(y = mindist[:,1])

fig.update_layout(title={'text':"<b>Protein-POPC atoms mindistance<b>",
                         'x':0.5,
                         'y':0.95,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis_title = "Frame number",
                  yaxis_title = "Distance [Å]")

fig.show()
make_image(fig, "protein_membrane_distance_gmx")

### Analysis of the protein angle with the z-axis
This analysis too was done in GROMACS with the <i>gangle</i> function, and here we load and plot the data for visualization.

In [ ]:
x = np.genfromtxt(rdir / "angle_results.xvg", usecols=0, skip_header=17)
y = 180 - np.genfromtxt(rdir / "angle_results.xvg", usecols=1, skip_header=17)

fig = px.line(y = y)

fig.update_layout(title={'text':"<b>Protein angle with respect to <i>z</i><b>", # prima era "protein orientation"
                         'x':0.5,
                         'y':0.95,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis_title = "Frame number",
                  yaxis_title = "Angle [degrees]")

fig.show()
make_image(fig, "protein_angle")

In [ ]:
# maximum value
print(f"Maximum value of {np.argmax(y)} reached at frame {y[np.argmax(y)]}")

# mean in the last 500 frames
print(f"The mean value in the last 500 frames is of {round(np.mean(y[5500:]),3)}")

These following blocks were used to understand how much the `z` parameter of the center of mass of all the POPC lipids was moving during the simulation. We were wondering if it would be best to compute the protein distance as the one between the "punctual" `z` value of the POP resiudes, or as the one between the average value of POPC height during the simulation.

In [ ]:
if not (simdir / "comparison.pickle").exists():
    lukG_al = mda.Universe(str(lukG_GRO), str(lukG_XTC_al))
    popz = []
    proz = []
    for ts in lukG.trajectory[:]:
        popz.append(lukG.select_atoms("resname POPC").center_of_mass()[2])
        proz.append(lukG.select_atoms("protein").center_of_mass()[2])
       
    data = [np.array(popz), np.array(proz)]

    with open(simdir / "comparison.pickle", "wb") as outf:
        pickle.dump(data, outf)
else:
    with open(simdir / "comparison.pickle", "rb") as infile:
        data = pickle.load(infile)

In [ ]:
diffz = difference_df.iloc[:, 2]
popz = data[0]
proz = data[1]

fig = go.Figure()
fig.add_trace(go.Line(y = popz, name = 'popc z'))  # z of POPC COM
fig.add_trace(go.Line(y = proz, name = 'prot z'))  # z of Prot COM
fig.add_trace(go.Line(y = diffz, name = 'diff z')) # z of difference
#fig.add_trace(go.Line(y = (proz - popz), name = 'comp diff'))

# add a line on the mean of the z of POPC residues
fig.add_hline(y = np.mean(popz))

# distance from mean of popc
fig.add_trace(go.Line(y = (proz - np.mean(popz)), name = 'diff prot to mean popc')) # z of difference

fig.update_layout(title={'text':"Protein-membrane distance from POPC by frame and by average",
                         'x':0.5,
                         'xanchor':'center',
                         'yanchor':'top'},
                  xaxis_title = "Frame number",
                  yaxis_title = "z-value of CoM")


fig.show()

make_image(fig, "distance_difference")

So, in the graph we have:
- In RED we have the z component of the center of mass of the protein
- In BLUE the z component of the center of mass of all the POPC lipids
- In GREEN the difference between the two z at each frame
- In VIOLET the difference between the z of the protein in every frame and the mean z of the POPC in the whole simulation